In [6]:
from Tkinter import *
from math import sin, cos, pi

# Class Sphero - robot object to be created (Done) %run Sphero.ipynb
# Class Walls to be created
# Class world to be created which contains the boundaries, walls (worls is a parent for walls, trajectory, entry and exit points)
# Class Sphero_trajectory - Creating the waypoint trajectory for the robot (child of room)
# Class 1D - Gaussian along the wall to represent the collision (this will become the child of walls)

class world(object):
    """ An artificial world is created for the SLAM problem. The other objects in the world are child classes in the artificial 
    world"""
    def draw(self, at_step):
        print "Object to be drawn at certain point of time:", at_step
        
    def background_draw(self):
        print "Background to be drawn"
        
world_size = (225.0, 225.0)
boundary_width = 5

class boundaries(world):
    
    def __init__(self, world_size, boundary_width):
        self.world_size = world_size



master = Tk()
w = Canvas(master,width=200,height=100)
w.pack()

w.create_line(0,0,200,100)
w.create_line(0,100,200,0,fill = "red", dash=(10,10))
w.create_rectangle(50,25,150,75, fill="blue")
mainloop()

In [27]:
from Tkinter import *
w = Canvas(Tk(),width=500,height=500)
w.pack()

w.create_oval(100,0,0,100,fill="red")
mainloop()